# Document Classification with Naive Bayes - Lab

## Introduction

In this lesson, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

In this lab you will:  

* Implement document classification using Naive Bayes

## Import the dataset

To start, import the dataset stored in the text file `'SMSSpamCollection'`.

In [10]:
# Your code here
import pandas as pd
df = pd.read_csv('SMSSpamCollection', sep='\t', names=['label', 'text'])
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Account for class imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [15]:
# Your code here
minority = df.loc[df['label'] == 'spam']
majority = df.loc[df['label'] == 'ham'].sample(n = len(minority))

df2 = pd.concat([minority, majority]).reset_index(drop = True)
df2

,label,text
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...
1,spam,FreeMsg Hey there darling it's been 3 week's n...
2,spam,WINNER!! As a valued network customer you have...
3,spam,Had your mobile 11 months or more? U R entitle...
4,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
1489,ham,Howz that persons story
1490,ham,I always chat with you. In fact i need money c...
1491,ham,The monthly amount is not that terrible and yo...
1492,ham,Er yep sure. Props?


## Train-test split

Now implement a train-test split on the dataset: 

In [16]:
# Your code here
from sklearn.model_selection import train_test_split
X = df2['text']
y = df2['label']
X_train, X_test, y_train, y_test = train_test_split(X, y)
train_df = pd.concat([X_train, y_train], axis = 1)
test_df = pd.concat([X_test, y_test], axis = 1)

## Create the word frequency dictionary for each class

Create a word frequency dictionary for each class: 

In [24]:
# Your code here
from functools import reduce
spam_all = reduce(lambda a, b: a + b, df2.loc[df2['label'] == 'spam', 'text'].values)
ham_all = reduce(lambda a, b: a + b, df2.loc[df2['label'] == 'ham', 'text'].values)

spam_bag = {}
for word in spam_all.split():
    spam_bag[word] = spam_bag.get(word, 0) + 1

ham_bag = {}
for word in ham_all.split():
    ham_bag[word] = ham_bag.get(word, 0) + 1
    
classes = ['spam','ham']
class_freq = dict(zip(classes, [spam_bag, ham_bag]))

## Count the total corpus words
Calculate V, the total number of words in the corpus: 

In [25]:
# Your code here
all_words = spam_all+ham_all
vocab = set()
for word in all_words.split():
    vocab.add(word)

V = len(vocab)
V

7661

## Create a bag of words function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [26]:
# Your code here
def bag_it(text):
    bag = {}
    for word in text.split():
        bag[word] = bag.get(word, 0) + 1
    return bag

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [34]:
# Your code here
import numpy as np
def classify_doc(doc, class_word_freq, p_classes, V):
    bag = bag_it(doc)
    
    priors = p_classes
    posteriors = []
    for prior, class_ in zip(priors, classes):
        for word in bag.keys():
            num = bag.get(word, 0)+1
            denom = class_word_freq.get(word, 0) + V
            prior *= (num/denom)
        posteriors.append(prior)
    return classes[np.argmax(posteriors)]

## Test your classifier

Finally, test your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [35]:
# Your code here

y_hat_train = X_train.map(lambda x: classify_doc(x, class_freq, [0.5,0.5], V))
residuals = y_train == y_hat_train
residuals.value_counts(normalize=True)

True     0.500893
False    0.499107
dtype: float64

## Level up (Optional)

Rework your code into an appropriate class structure so that you could easily implement the algorithm on any given dataset.

## Summary

Well done! In this lab, you practiced implementing Naive Bayes for document classification!